In [1]:
import pandas as pd
import re
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import string
import nltk
%matplotlib inline
from sklearn.decomposition import SparsePCA
from sklearn.preprocessing import MaxAbsScaler, StandardScaler

In [2]:
mabs = MaxAbsScaler()

In [3]:
ss =  StandardScaler()

In [4]:
pca = SparsePCA(100, random_state = 1)

In [5]:
df_train= pd.read_csv('C:/Users/Dell/Downloads/train_E6oV3lV.csv')
df_train.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [6]:
df_test = pd.read_csv('C:/Users/Dell/Downloads/test_tweets_anuFYb8.csv')
df_test.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."


In [7]:
comb = df_train.append(df_test, ignore_index=True)


In [8]:
comb['tweet'] = comb['tweet'].apply(lambda x: " ".join(x.lower() for x in x.split()))
comb['tweet'].head()

0    @user when a father is dysfunctional and is so...
1    @user @user thanks for #lyft credit i can't us...
2                                  bihday your majesty
3    #model i love u take with u all the time in ur...
4                  factsguide: society now #motivation
Name: tweet, dtype: object

In [9]:
comb['tweet'] = comb['tweet'].str.replace("[^a-zA-Z#]", " ") #removing special characters
comb.head()

,id,label,tweet
0,1,0.0,user when a father is dysfunctional and is so...
1,2,0.0,user user thanks for #lyft credit i can t us...
2,3,0.0,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ur...
4,5,0.0,factsguide society now #motivation


In [10]:
comb['tweet'] = comb['tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3])) #removing stop words
comb.head()

,id,label,tweet
0,1,0.0,user when father dysfunctional selfish drags k...
1,2,0.0,user user thanks #lyft credit cause they offer...
2,3,0.0,bihday your majesty
3,4,0.0,#model love take with time
4,5,0.0,factsguide society #motivation


In [11]:
token_tweet = comb['tweet'].apply(lambda x: x.split()) #tokenization is the process of splitting a string of text into tokens.
token_tweet.head()

0    [user, when, father, dysfunctional, selfish, d...
1    [user, user, thanks, #lyft, credit, cause, the...
2                              [bihday, your, majesty]
3                     [#model, love, take, with, time]
4                   [factsguide, society, #motivation]
Name: tweet, dtype: object

In [12]:
from nltk.stem.porter import *
stemmer = PorterStemmer()

token_tweet = token_tweet.apply(lambda x: [stemmer.stem(i) for i in x]) # stemming
token_tweet.head()

0    [user, when, father, dysfunct, selfish, drag, ...
1    [user, user, thank, #lyft, credit, caus, they,...
2                              [bihday, your, majesti]
3                     [#model, love, take, with, time]
4                         [factsguid, societi, #motiv]
Name: tweet, dtype: object

In [13]:
for i in range(len(token_tweet)):
    token_tweet[i] = ' '.join(token_tweet[i])

comb['tweet'] = token_tweet
comb['tweet'].head() 

0    user when father dysfunct selfish drag kid int...
1    user user thank #lyft credit caus they offer w...
2                                  bihday your majesti
3                           #model love take with time
4                             factsguid societi #motiv
Name: tweet, dtype: object

In [14]:
comb['tweet'].shape

(49159,)

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
# TF-IDF feature matrix  ## TF-IDF works by penalizing the common words by assigning them lower weights while giving importance to words which are rare 
#in the entire corpus but appear in good numbers in few documents.
tfidf = tfidf_vectorizer.fit_transform(comb['tweet'])
tfidf

<49159x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 208694 stored elements in Compressed Sparse Row format>

In [16]:
tfidf.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score


In [18]:
31962/49159

0.6501759596411644

In [19]:
tfidf_arr = tfidf.toarray()

In [20]:
train_tfidf = tfidf_arr[:31962,:]
test_tfidf = tfidf_arr[31962:,:]


In [21]:
xtrain_tfidf, xvalid_tfidf, ytrain, yvalid = train_test_split(train_tfidf, df_train['label'],  
                                                          random_state=42, 
                                                          test_size=0.3)

In [22]:
xtrain_tfidf = mabs.fit_transform(xtrain_tfidf)
xvalid_tfidf =  mabs.fit_transform(xvalid_tfidf)

In [23]:
#xtrain_tfidf = pca.fit_transform(xtrain_tfidf)
#xvalid_tfidf =  pca.fit_transform(xvalid_tfidf)

In [23]:
lreg = LogisticRegression()

lreg.fit(xtrain_tfidf, ytrain)

prediction = lreg.predict_proba(xvalid_tfidf)
prediction_int = prediction[:,1] >= 0.3
prediction_int = prediction_int.astype(np.int)

f1_score(yvalid, prediction_int)

0.535472972972973

In [24]:
from sklearn.ensemble import RandomForestClassifier


In [ ]:
rf = RandomForestClassifier(n_estimators=400, random_state=11).fit(xtrain_tfidf, ytrain)

prediction = rf.predict(xvalid_tfidf)
f1_score(yvalid, prediction)

In [27]:
from sklearn import svm

svc = svm.SVC(kernel='linear', C=1, probability=True).fit(xtrain_tfidf, ytrain)

prediction = svc.predict_proba(xvalid_tfidf)
prediction_int = prediction[:,1] >= 0.3
prediction_int = prediction_int.astype(np.int)
f1_score(yvalid, prediction_int)

0.513145965548504

In [ ]:
test_pred = rf.predict(test_tfidf)
df_test['label'] = test_pred
submission = df_test[['id','label']]
submission.to_csv('gett.csv', index=False)

In [ ]:
#from xgboost import XGBClassifier

#xgb = XGBClassifier(max_depth=6, n_estimators=1000).fit(xtrain_tfidf, ytrain)

#prediction = xgb.predict(xvalid_tfidf)
#f1_score(yvalid, prediction)